# Encryption

![Encryption explained](https://cloudsafari.blob.core.windows.net/blogresources/encryption.png)

![Key hierarchy](https://cloudsafari.blob.core.windows.net/blogresources/encryption-hierarchy-stack.gif)

**Transparent Database Encryption (TDE)**

Protect only files or backups

Create a new database and fill it with data from Adventureworks database

In [ ]:
CREATE DATABASE TDESample;
GO

USE TDESample;
GO

--import some tables from the AdventureWorks database

SELECT * 
INTO dbo.customers
FROM AdventureWorks.Sales.Customer;
GO

SELECT * 
INTO dbo.Products
FROM AdventureWorks.Production.Product;
GO

SELECT *
INTO dbo.SalesOrders
FROM AdventureWorks.Sales.SalesOrderHeader;
GO

SELECT *
INTO dbo.SalesDetails
FROM AdventureWorks.Sales.SalesOrderDetail;
GO


Check database status

In [ ]:
select database_id,name,is_encrypted from sys.databases
go
select * from sys.certificates
go
select * from sys.dm_database_encryption_keys --shows the tempdb encrypted as well.
go
--get which certificates used by which DEKs
select c.name as certificateName,dek.database_id,
DB_NAME(dek.database_id) as 'Database Name',dek.encryption_state
from sys.dm_database_encryption_keys dek 
join sys.certificates c on dek.encryptor_thumbprint=c.thumbprint

Encrypt the database

In [ ]:

--service master key is already created
--let's create the database master key of the master database
use master
go
SELECT * FROM sys.symmetric_keys;





Create master database's master key

The database master key is a symmetric key that is used to protect the private keys of certificates and asymmetric keys that are present in the database. It can also be used to encrypt data, but it has length limitations that make it less practical for data than using a symmetric key. To enable the automatic decryption of the database master key, a copy of the key is encrypted by using the SMK. It is stored in both the database where it is used and in the master system database.

In [ ]:
use master;
GO
create master key encryption by password='MasterDBPassword!'
GO
SELECT * FROM sys.symmetric_keys;


Create the certificate to use to encrypt the DEK

In [ ]:
--create certificate to use it to encrypt the AdventureWorks Databaes Encryption Key (don't encrypt by password)
Create certificate cert_AW_encryptDEK authorization dbo
with subject='the DEK certificate for AdventureWorks database'
go



--now move to TDESample and create the DEK

TDE performs real-time I/O encryption and decryption of the data and log files. The encryption uses a database encryption key (DEK), which is stored in the database boot record for availability during recovery. The DEK is a symmetric key secured by using a certificate stored in the master database of the server or an asymmetric key protected by an EKM module

In [ ]:
use TDESample;
go
Create database encryption key 
with Algorithm=AES_256
Encryption by server certificate cert_aw_encryptDEK
go
Alter database TDESample
set encryption ON
go

Checking...

In [ ]:
USE master;
GO
select database_id,name,is_encrypted from sys.databases


In [ ]:
use master;
go
select * from sys.certificates


In [ ]:
use master;
go
select * from sys.dm_database_encryption_keys --shows the tempdb encrypted as well.


In [ ]:
use master;
go
--get which certificates used by which DEKs
select c.name as certificateName,dek.database_id,
DB_NAME(dek.database_id) as 'Database Name',dek.encryption_state
from sys.dm_database_encryption_keys dek 
join sys.certificates c on dek.encryptor_thumbprint=c.thumbprint

Now, let's try backup the database 

In [ ]:
use master
go
BACKUP DATABASE [TDESample] 
TO  DISK = N'C:\Temp\TDESample_afterTDE.bak' 
WITH NAME = N'TDESample-Full Database Backup'
GO
--backup certificate
use master
go
backup certificate cert_aw_encryptDEK
to file='c:\Temp\cert_aw_encryptDEK.cert'
with private key(file='c:\Temp\cert_aw_encryptDEK_privateKey.cert',
encryption by password='privateKeyPass!')--have to use password for private key file




**connect to another server**

restore the certificate first


In [ ]:

use master
go
SELECT * FROM sys.symmetric_keys;

--this step needed only if the master database in the new server doesn't have master key
create master key encryption by password='MasterDBPassword!'
go
-------------------------------------------------------------------

create certificate cert_aw_encryptDEK
from file='c:\Temp\cert_aw_encryptDEK.cert'
with private key(file='c:\Temp\cert_aw_encryptDEK_privateKey.cert', decryption by password='privateKeyPass!')
go

RESTORE DATABASE [TDESample] 
FROM  DISK = N'C:\Temp\TDESample_afterTDE.bak' 
WITH MOVE N'TDESample' TO N'C:\Program Files\Microsoft SQL Server\MSSQL10.SQLDEV02\MSSQL\DATA\TDESample.mdf',
MOVE N'TDESample_Log' TO N'C:\Program Files\Microsoft SQL Server\MSSQL10.SQLDEV02\MSSQL\DATA\TDESample_1.ldf'
GO

# CLEAN UP


In [ ]:
use master;
GO
DROP DATABASE TDESample;
GO
DROP Certificate cert_AW_encryptDEK;
GO
drop master key 